In [23]:
import pandas as pd
import Levenshtein as levenshtein
import json
import string
import ast

In [8]:
# Movie Info
df_tmdb_movie = pd.read_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/tmdb/tmdb.csv")
df_tmdb_movie.head()

,Unnamed: 0,id,title,overview,popularity,vote_average,vote_count,release_date,keywords,genres,cast,crew
0,0,19404,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...",31.222,8.7,3323,1995-10-20,[],"['Comedy', 'Drama', 'Romance']","['Shah Rukh Khan', 'Kajol', 'Amrish Puri', 'An...",['Aditya Chopra']
1,1,278,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,76.654,8.7,20434,1994-09-23,"['prison', 'corruption', 'police brutality', '...","['Drama', 'Crime']","['Tim Robbins', 'Morgan Freeman', 'Bob Gunton'...",['Frank Darabont']
2,2,238,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",75.306,8.7,15270,1972-03-14,"['italy', 'loss of loved one', 'love at first ...","['Drama', 'Crime']","['Marlon Brando', 'Al Pacino', 'James Caan', '...",['Francis Ford Coppola']
3,3,724089,Gabriel's Inferno Part II,Professor Gabriel Emerson finally learns the t...,21.501,8.6,1369,2020-07-31,['based on novel or book'],['Romance'],"['Melanie Zanetti', 'Giulio Berruti', 'James A...",['Tosca Musk']
4,4,424,Schindler's List,The true story of how businessman Oskar Schind...,40.585,8.6,12202,1993-11-30,"['based on novel or book', 'factory', 'concent...","['Drama', 'History', 'War']","['Liam Neeson', 'Ben Kingsley', 'Ralph Fiennes...",['Steven Spielberg']


In [4]:
df_tmdb_movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9480 entries, 0 to 9479
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    9480 non-null   int64  
 1   id            9480 non-null   int64  
 2   title         9480 non-null   object 
 3   overview      9464 non-null   object 
 4   popularity    9480 non-null   float64
 5   vote_average  9480 non-null   float64
 6   vote_count    9480 non-null   int64  
 7   release_date  9480 non-null   object 
 8   keywords      9480 non-null   object 
 9   genres        9480 non-null   object 
 10  cast          9480 non-null   object 
 11  crew          9480 non-null   object 
dtypes: float64(2), int64(3), object(7)
memory usage: 888.9+ KB


Kết quả Schema Matching: (Những trường bị đánh * trong Source Schema là bị matching sai)

- Bảng Movie:

| Source Schema | Data warehouse Schema |
|---------------|-----------------------|
| title         | movie_title           |
| keywords*     | content_rating        |
| genres        | genres                |
| crew          | directors             |
| cast          | production_company    |
| actor         | actors                |
| vote_average* | runtime               |
| overview      | movie_info            |
| release_date  | release_date          |

- Bảng Rating:

| Source Schema | Data warehouse Schema |
|---------------|-----------------------|
| vote_average  | avg_rating            |
| release_date* | rating_count          |
| overview*     | link                  |

Nhận xét và hướng xử lý của các trường:

- Bảng Movie:
    + Trường genres, cast, crews trong TMDB đang được viết theo dạng [item1, item2, ...], trong khi dạng của các trường đó trong data warehouse là item1, item2, ..., đồng thời các giá trị các trường trong TMDB có thể khác so với trong Data Warehouse => cần phải biến đổi lại
    + Trường content_rating và runtime không có trong nguồn TMDB
- Bảng Rating:
    + Kiểm tra lại dữ liệu movie, ta nhận thấy trường rating_count trong data warehouse được thể hiện bằng trường vote_count trong TMDB
    + Trường link không có trong nguồn TMDB

# Xử lý trường genres

In [9]:
# Chuyển string thành list
def string_to_list(stri):
    return ast.literal_eval(stri)

df_tmdb_movie["genres"]=[string_to_list(i) for i in df_tmdb_movie["genres"]]

# Lấy các giá trị riêng biệt trong trường genres
genres_list = []
for i in df_tmdb_movie["genres"]:
    genres_list+=i
genres_list = list(set(genres_list))
genres_list

['Adventure',
 'Horror',
 'Western',
 'War',
 'Drama',
 'Family',
 'Fantasy',
 'Thriller',
 'Music',
 'Mystery',
 'TV Movie',
 'Romance',
 'Comedy',
 'History',
 'Animation',
 'Action',
 'Science Fiction',
 'Crime']

In [12]:
count=0
for i in df_tmdb_movie["genres"]:
    if len(i)==0:
        count+=1
print("Số bản ghi trong nguồn TMDB mà trống thể loại: "+str(count))

Số bản ghi trong nguồn TMDB mà trống thể loại: 2


Có 2 bản ghi trong TMDB bị trống thể loại, ta sẽ chuyển về Null

In [10]:
warehouse_genres_list = list(pd.read_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/warehouse/field_value/genres.csv")["genres"])
warehouse_genres_list

['Gay & Lesbian',
 'Documentary',
 'Horror',
 'History',
 'Comedy',
 'War',
 'Crime',
 'Western',
 'IMAX',
 'Faith & Spirituality',
 'Biography',
 'Cult Movies',
 'Musical & Performing Arts',
 'Thrill',
 'Drama',
 'Kids & Family',
 'Romance',
 'Special Interest',
 'Film-Noir',
 'Action & Adventure',
 'Mystery & Suspense',
 'Television',
 'Science Fiction & Fantasy',
 'Animation',
 'Art House & International',
 'Classics',
 'Sports & Fitness',
 'Anime & Manga']

Từ 2 danh sách thể loại, ta có thể mapping như dưới:
|Data warehouse Genres|TMDB Genres |
| :- | :- |
|‘Classic’||
|‘Documentary’||
|‘Western’|‘Western’|
|‘Horror’|‘Horror’|
|‘Science Fiction & Fantasy’\*|‘Fantasy’, ‘Science Fiction’|
|‘Gay & Lesbian’||
|‘Drama’|‘Drama’|
|‘Comedy’|‘Comedy’|
|‘Cult Movies’||
|‘Romance’|‘Romance’|
|‘Television’||
|‘Sport & Fitness’||
|‘Art House & International’||
|‘Special Interest’||
|‘Animation’|‘Animation’|
|‘Musical & Performing Arts’\*|‘Music’|
|‘Faith & Spirituality’||
|‘Mystery & Suspense’\*|‘Mystery’|
|‘Anime & Manga’||
|‘Action & Adventure’\*|‘Adventure’, ‘Action’|
|‘Kids & Family’\*|‘Family’|
|‘War’|‘War’|
|‘Crime’|‘Crime’|
|‘Film-Noir’||
|‘IMAX’||
|‘Thrill’\*|‘Thriller’|
|‘Biography’||
|‘History’|‘History’|

Những giá trị được đánh * là những giá trị cần phải chuyển đổi

Giá trị như TV Movie ta sẽ thêm vào Data Warehouse

In [13]:
replace_genres_dict = {'Mystery' : 'Mystery & Suspense', 'Thriller': 'Thrill',
                       'Fantasy': 'Science Fiction & Fantasy', 'Science Fiction': 'Science Fiction & Fantasy',
                       'Adventure': 'Action & Adventure','Action': 'Action & Adventure',
                       'Music': 'Musical & Performing Arts',
                       'Family': 'Kids & Family'}

def preprocess_genres(genre):
    if len(genre)==0:
        return None
    else:
        new_list_genres = ""
        for idx, i in enumerate(genre):
            if i in replace_genres_dict:
                new_list_genres+=replace_genres_dict[i]
            else:
                new_list_genres+=i
            if idx != len(genre)-1:
                new_list_genres+=", "
        return new_list_genres
df_tmdb_movie["genres"] = [preprocess_genres(i) for i in df_tmdb_movie["genres"]]

In [14]:
# Thêm các giá trị mới vào trong genres warehouse
warehouse_genres_list+=["TV Movie"]
warehouse_genres_list = list(set(warehouse_genres_list))
pd.DataFrame(warehouse_genres_list, columns=["genres"]).to_csv(
    "/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/warehouse/field_value/genres.csv",
    index=False 
)

# Xử lý trường cast và crew

In [15]:
# Chuyển string thành list
df_tmdb_movie["cast"]=[string_to_list(i) for i in df_tmdb_movie["cast"]]
df_tmdb_movie["crew"]=[string_to_list(i) for i in df_tmdb_movie["crew"]]

In [17]:
# Lấy các giá trị riêng biệt trong trường cast và crew
cast_list = []
for i in df_tmdb_movie["cast"]:
    cast_list+=i
cast_list = list(set(cast_list))

crew_list = []
for i in df_tmdb_movie["crew"]:
    crew_list+=i
crew_list = list(set(crew_list))

print("Số giá trị riêng biệt trong trường cast: "+str(len(cast_list)))
print("Số giá trị riêng biệt trong trường crew: "+str(len(crew_list)))

Số giá trị riêng biệt trong trường cast: 25212
Số giá trị riêng biệt trong trường crew: 3868


In [22]:
warehouse_directors_actors_list = list(pd.read_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/warehouse/field_value/directors_actors.csv")["name"])
print("Số giá trị của trường directors và actors riêng biệt có trong warehouse: "+ str(len(warehouse_directors_actors_list)))

Số giá trị của trường directors và actors riêng biệt có trong warehouse: 213394


Xử lý tương tự như trường director và crew trơng nguồn tmdb, ta cũng sẽ sử dụng string matching + lọc các ứng cử viên theo chữ cái đầu tiên

## Trường cast

In [21]:
def get_candidate(source, first_character):
    if first_character in string.punctuation:
        return source
    else:
        candidate = []
        for i in source:
            if type(i)==str:
                if i[0].lower() == first_character:
                    candidate.append(i)
                elif i[0] in string.punctuation:
                    candidate.append(i)
        return candidate

In [24]:
cast_mapping = {}
number_of_case = []
for idx, i in enumerate(warehouse_directors_actors_list):
    if idx % 30000 == 0:
        print(idx)
    candidate = get_candidate(cast_list, i[0].lower())
    number_of_case.append(len(candidate))
    match=[]
    for j in candidate:
        if levenshtein.ratio(i, j)>=0.9:
            match.append(j)
    cast_mapping[i]=match

0
30000
60000
90000
120000
150000
180000
210000


In [26]:
count=0
for key, value in cast_mapping.items():
    if len(value)!=0:
        count+=1
        print("actors (Data warehouse): "+key)
        print("cast (TMDB): "+str(value))
        print()
        if count==10:
            break

actors (Data warehouse): Robin Givens
cast (TMDB): ['Robin Givens']

actors (Data warehouse): Philippe Caubère
cast (TMDB): ['Philippe Caubère']

actors (Data warehouse): Dean Shek
cast (TMDB): ['Dean Shek']

actors (Data warehouse): Billy Rieck
cast (TMDB): ['Billy Rieck']

actors (Data warehouse): Joleigh Fioreavanti
cast (TMDB): ['Joleigh Fioravanti']

actors (Data warehouse): Wayne Duvall
cast (TMDB): ['Wayne Duvall']

actors (Data warehouse): Miranda Edwards
cast (TMDB): ['Miranda Edwards']

actors (Data warehouse): Louise Bourgoin
cast (TMDB): ['Louise Bourgoin']

actors (Data warehouse): Jason Gould
cast (TMDB): ['Jason Gould']

actors (Data warehouse): Sari Lennick
cast (TMDB): ['Sari Lennick']



In [36]:
cast_mapping_dict = {}
for key, value in cast_mapping.items():
    if len(value)!=0:
        for i in value:
            cast_mapping_dict[i]=key
# Lưu lại các giá trị cast (actor) chưa có trong warehouse 
remain_cast = [item for item in cast_list if item not in list(cast_mapping_dict.keys())]

In [37]:
# Chuyển đổi giá trị trường cast trong tmdb về giá trị trường actors trong data warehouse
def preprocess_name(name, replace_name_dict):
    if len(name)==0:
        return None
    else:
        new_list_name = ""
        for idx, i in enumerate(name):
            if i in replace_name_dict:
                new_list_name+=replace_name_dict[i]
            else:
                new_list_name+=i
            if idx != len(name)-1:
                new_list_name+=", "
        return new_list_name
df_tmdb_movie["cast"] = [preprocess_name(i, cast_mapping_dict) for i in df_tmdb_movie["cast"]]

## Trường crew

In [30]:
crew_mapping = {}
number_of_case = []
for idx, i in enumerate(warehouse_directors_actors_list):
    if idx % 30000 == 0:
        print(idx)
    candidate = get_candidate(crew_list, i[0].lower())
    number_of_case.append(len(candidate))
    match=[]
    for j in candidate:
        if levenshtein.ratio(i, j)>=0.9:
            match.append(j)
    crew_mapping[i]=match

0
30000
60000
90000
120000
150000
180000
210000


In [35]:
count=0
for key, value in crew_mapping.items():
    if len(value)!=0:
        count+=1
        print("directors (Data warehouse): "+key)
        print("crew (TMDB): "+str(value))
        print()
        if count==10:
            break

directors (Data warehouse): Catalina Aguilar Mastretta
crew (TMDB): ['Catalina Aguilar Mastretta']

directors (Data warehouse): Roberto Rossellini
crew (TMDB): ['Roberto Rossellini']

directors (Data warehouse): Yves Robert
crew (TMDB): ['Yves Robert']

directors (Data warehouse): Nick Ormerod
crew (TMDB): ['Nick Ormerod']

directors (Data warehouse): John Maclean
crew (TMDB): ['John Maclean']

directors (Data warehouse): Jalil Lespert
crew (TMDB): ['Jalil Lespert']

directors (Data warehouse): Andrew Dominik
crew (TMDB): ['Andrew Dominik']

directors (Data warehouse): Kevin Williamson
crew (TMDB): ['Kevin Williamson']

directors (Data warehouse): Kirsten Sheridan
crew (TMDB): ['Kirsten Sheridan']

directors (Data warehouse): Fritz Kiersch
crew (TMDB): ['Fritz Kiersch']



In [38]:
crew_mapping_dict = {}
for key, value in crew_mapping.items():
    if len(value)!=0:
        for i in value:
            crew_mapping_dict[i]=key
# Lưu lại các giá trị crew (director) chưa có trong warehouse 
remain_crew = [item for item in crew_list if item not in list(crew_mapping_dict.keys())]

In [41]:
# Chuyển đổi giá trị trường crew trong tmdb về giá trị trường directors trong data warehouse
df_tmdb_movie["crew"] = [preprocess_name(i, crew_mapping_dict) for i in df_tmdb_movie["crew"]]

## Lưu các giá trị director (crew) và actor (cast) mới vào trong data warehouse

In [25]:
# Lưu cast mapping và crew mapping
with open('/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/tmdb/mapping/cast_mapping.json', "w") as write_file:
    json.dump(cast_mapping, write_file, indent=4)
with open('/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/tmdb/mapping/crew_mapping.json', "w") as write_file:
    json.dump(crew_mapping, write_file, indent=4)

In [42]:
warehouse_directors_actors_list+=remain_cast
warehouse_directors_actors_list+=remain_crew
warehouse_directors_actors_list=list(set(warehouse_directors_actors_list))
pd.DataFrame(warehouse_directors_actors_list, columns=["name"]).to_csv(
    "/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/warehouse/field_value/directors_actors.csv",
    index=False 
)

## Loại các trường không cần thiết và Lưu dữ liệu chuyển đổi

In [45]:
df_tmdb_final = df_tmdb_movie[['id', 'title', 'overview', 'vote_average', 'vote_count', 
                               'release_date', 'genres', 'cast', 'crew']]

In [47]:
df_tmdb_final.to_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/tmdb/movies_preprocess.csv", index=False)